In [71]:
import torch
import seaborn as sea
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from game_log import load_current_line, GameLog
plt.rcParams['figure.figsize'] = [15, 8]
pd.set_option("display.precision", 3)

In [72]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.min_rows', 10)
torch.set_printoptions(sci_mode=False)

In [73]:
opp_stats = pd.read_csv('opponent_stats.csv',index_col=0)
schedule = pd.read_csv('schedule.csv',index_col=0)
KM_vals = pd.read_csv('KM_vals.csv',index_col=0)

today = dt.datetime.today()
year = today.year
month = today.month
day = today.day
today_str = f'{year}_{month}_{day}'


pp_path = f'Lines/pp/pp_{today_str}.csv'
unabated_path = f'Lines/unabated/unabated_{today_str}.csv'

#pp_lines = load_current_line(pp_path,'prop_id','time')
#unabated = load_current_line(unabated_path,'prop_id','time')

In [74]:
pdata = pd.read_csv('pdata.csv')
pdata['season'] = pdata['date'].apply(lambda x: x.split('-')[0])
data = pd.read_csv('data_2024.csv')
data23 = pd.read_csv('data_2023.csv')
data = pd.concat((pdata,data,data23))
data= data.sort_values(by=['player','date']).reset_index(drop=True)
data.sample(3)

,player,G,date,series,team,H/A,opp,G#,W/L,GS,...,BLK,TOV,PF,PTS,GmSc,+/-,pos,KM,season,age
27943,P.J. Washington,34,2022-12-27,NaN,CHA,0,GSW,NaN,-5,1,...,1,2,3,14,7.0,-2.0,PF,15,2023,24-126
9208,Doug McDermott,44,2023-01-28,NaN,SAS,1,PHX,NaN,-10,0,...,0,3,4,10,2.0,-9.0,SF,15,2023,31-025
6471,Dario Saric,43,2023-03-01,NaN,OKC,1,LAL,NaN,-6,0,...,0,1,3,13,8.8,-4.0,PF-C,15,2023,28-327


In [75]:
gl = GameLog(data)

In [76]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
def pt_corr(player_a_name, player_b_name, team,stat1='PTS',stat2='PTS'):
    team_games = data.loc[data.team==team]
    # extract two columns of interest (i.e. points scored by the two players)
    player_a_dates = set(team_games[team_games['player'] == player_a_name]['date'])
    player_b_dates = set(team_games[team_games['player'] == player_b_name]['date'])
    common_dates = player_a_dates.intersection(player_b_dates)
    stats = team_games[team_games['date'].isin(common_dates)]
    
    player_a_points = stats[stats['player'] == player_a_name][stat1].reset_index(drop=True)
    player_b_points = stats[stats['player'] == player_b_name][stat2].reset_index(drop=True)
    sample = len(player_a_points)
    # calculate the correlation coefficient between the two columns
    corr_coeff = player_a_points.corr(player_b_points)
    # return the correlation coefficient
    return corr_coeff,sample
pt_corr('Stephen Curry','Klay Thompson','GSW')

(0.03776803308315927, 190)

In [77]:
team_list = data.team.unique()

In [78]:
score_avg = data.groupby('player')['PTS'].mean().reset_index().sort_values(by='PTS',ascending=False)
top_scorers = score_avg.player.unique()[0:125]

In [79]:
team2players = {}
for team in team_list:
    team_games = data.loc[data.team==team]
    players = team_games.loc[team_games.GS==1].player.unique()
    team2players[team]=players

In [178]:
stats = ['PTS','AST','TRB']
def get_correlations(team_list,player_list,stat1_list,stat2_list):
    correlate = []
    for team in team_list:
        for p in team2players[team]:
            if p in player_list:
                for p2 in team2players[team]:
                    if (p != p2) and (p2 in player_list):
                        for stat1 in stat1_list:
                            for stat2 in stat2_list:
                                corr,sample = pt_corr(p,p2,team,stat1,stat2)
                                correlate.append([team,sample,p,p2,stat1,corr,stat2])
                                corr,sample = pt_corr(p2,p,team,stat2,stat1)
                                correlate.append([team,sample,p2,p,stat2,corr,stat2])
    all_corr = pd.DataFrame(correlate,columns=['Team','Sample','P1','P2','Stat1','Corr','stat2']).sort_values(by='Corr').dropna()
    all_corr = all_corr.loc[all_corr['Sample']>10]
    return all_corr
all_corr = get_correlations(team_list,top_scorers,stats,stats)

c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\nump

In [ ]:
tonight = ['TOR','PHI','DET','NOP','ORL','UTA','PHO','SAS']
all_corr.reset_index(drop=True).drop_duplicates(inplace=True)
all_corr.loc[(all_corr.Sample > 20) & (all_corr.Team.isin(['PHI']))].tail(10)

,Team,Sample,P1,P2,Stat1,Corr,stat2
11613,PHI,77,Joel Embiid,Tyrese Maxey,AST,0.178,AST
11328,PHI,77,Joel Embiid,Tyrese Maxey,AST,0.178,PTS
11254,PHI,64,Joel Embiid,James Harden,PTS,0.199,TRB
11065,PHI,64,Joel Embiid,James Harden,PTS,0.199,PTS
11255,PHI,64,James Harden,Joel Embiid,TRB,0.199,TRB
11064,PHI,64,James Harden,Joel Embiid,TRB,0.199,PTS
11252,PHI,64,Joel Embiid,James Harden,PTS,0.464,AST
11253,PHI,64,James Harden,Joel Embiid,AST,0.464,AST
11058,PHI,64,James Harden,Joel Embiid,AST,0.464,PTS
11059,PHI,64,Joel Embiid,James Harden,PTS,0.464,PTS


In [181]:
all_corr.loc[all_corr.P1== 'Brook Lopez'].head(10)

,Team,Sample,P1,P2,Stat1,Corr,stat2
64,MIL,88,Brook Lopez,Khris Middleton,AST,-0.203,TRB
375,MIL,88,Brook Lopez,Khris Middleton,AST,-0.203,AST
50,MIL,104,Brook Lopez,Jrue Holiday,TRB,-0.188,AST
281,MIL,104,Brook Lopez,Jrue Holiday,TRB,-0.188,TRB
62,MIL,88,Brook Lopez,Khris Middleton,AST,-0.159,AST
369,MIL,88,Brook Lopez,Khris Middleton,AST,-0.159,AST
28,MIL,124,Brook Lopez,Giannis Antetokounmpo,AST,-0.143,TRB
195,MIL,124,Brook Lopez,Giannis Antetokounmpo,AST,-0.143,AST
18,MIL,124,Brook Lopez,Giannis Antetokounmpo,PTS,-0.143,PTS
181,MIL,124,Brook Lopez,Giannis Antetokounmpo,PTS,-0.143,PTS


In [ ]:
#all_corr.to_csv('reference_data/correlation.csv',index=0)

In [89]:
mask1 = team_games['pos'] == 'PG'
mask2 = team_games['pos'] == 'SG'

team_games = data.loc[(data.GS==1) & (data.team=='GSW')]
# extract two columns of interest (i.e. points scored by the two players)
player_a_dates = set(team_games[mask1]['date'])
player_b_dates = set(team_games[mask2]['date'])
common_dates = player_a_dates.intersection(player_b_dates)
stats = team_games[team_games['date'].isin(common_dates)]
# return the correlation coefficient

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_32848\78216638.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  player_a_dates = set(team_games[mask1]['date'])


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [94]:
data.groupby(['pos','GS'])[['MP','PTS']].mean().reset_index()

,pos,GS,MP,PTS
0,C,0,13.109,5.163
1,C,1,29.348,14.139
2,F,0,3.725,0.750
3,PF,0,15.859,5.999
4,PF,1,32.166,16.094
...,...,...,...,...
17,SF-SG,0,11.760,2.848
18,SF-SG,1,33.488,16.848
19,SG,0,16.583,6.952
20,SG,1,32.012,16.358


In [99]:
starters = data.loc[(data.team=='GSW') & (data.GS==1)].sort_values(by='date')

In [171]:
position_list = data.pos.unique()
position_list = ['PG','SG','SF','PF','C']

In [172]:
def get_pos(position,df):
    return df.loc[df.pos ==position]
get_pos(position_list[0],starters)

,player,G,date,series,team,H/A,opp,G#,W/L,GS,...,TOV,PF,PTS,GmSc,+/-,pos,KM,season,age,rest
4555,Chris Paul,1,2008-04-19,WC1,NOH,1,DAL,1.0,12,1,...,1,2,35,35.3,16.0,PG,15,2008,NaN,5.0
4556,Chris Paul,2,2008-04-22,WC1,NOH,1,DAL,2.0,24,1,...,3,5,32,35.4,24.0,PG,15,2008,NaN,3.0
4557,Chris Paul,3,2008-04-25,WC1,NOH,0,DAL,3.0,-10,1,...,1,5,16,10.7,-17.0,PG,15,2008,NaN,3.0
4558,Chris Paul,4,2008-04-27,WC1,NOH,0,DAL,4.0,13,1,...,1,3,16,14.7,12.0,PG,15,2008,NaN,2.0
4559,Chris Paul,5,2008-04-29,WC1,NOH,1,DAL,5.0,5,1,...,0,2,24,29.7,11.0,PG,15,2008,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4573,Chris Paul,2,2011-04-20,WC1,NOH,0,LAL,2.0,-9,1,...,1,4,20,17.3,-7.0,PG,15,2011,NaN,3.0
4574,Chris Paul,3,2011-04-22,WC1,NOH,1,LAL,3.0,-14,1,...,5,2,22,19.4,-12.0,PG,15,2011,NaN,2.0
4575,Chris Paul,4,2011-04-24,WC1,NOH,1,LAL,4.0,5,1,...,5,2,27,31.4,9.0,PG,15,2011,NaN,2.0
4576,Chris Paul,5,2011-04-26,WC1,NOH,0,LAL,5.0,-16,1,...,4,1,20,19.8,-7.0,PG,15,2011,NaN,2.0


In [173]:
correlation_list = []
stat = 'PTS'
for team in team_list:
    starters = data.loc[(data.team==team) & (data.GS==1)].sort_values(by='date')
    for pos1 in position_list:
        for pos2 in position_list:
            a = get_pos(pos1,starters)[['PTS','date']]
            b = get_pos(pos2,starters)[['PTS','date']]
            player_a_dates = set(a['date'])
            player_b_dates = set(b['date'])

            common_dates = player_a_dates.intersection(player_b_dates)
            stats = starters[starters['date'].isin(common_dates)]
            player_a_points = a[a['date'].isin(common_dates)][stat].reset_index(drop=True)
            player_b_points = b[b['date'].isin(common_dates)][stat].reset_index(drop=True)
            correlation = player_a_points.corr(player_b_points)
            correlation_list.append((pos1,pos2,correlation,len(common_dates),team))
    x = pd.DataFrame(correlation_list)
x


,0,1,2,3,4
0,PG,PG,1.000,138,MIL
1,PG,SG,-0.017,95,MIL
2,PG,SF,-0.112,107,MIL
3,PG,PF,0.013,138,MIL
4,PG,C,0.116,125,MIL
...,...,...,...,...,...
770,C,PG,NaN,0,NOH
771,C,SG,NaN,0,NOH
772,C,SF,NaN,0,NOH
773,C,PF,NaN,0,NOH


In [174]:
x = x.dropna().sort_values(by=2)
x

,0,1,2,3,4
692,PF,SF,-1.000,2,UTA
688,SF,PF,-1.000,2,UTA
632,SG,SF,-0.555,71,TOR
636,SF,SG,-0.555,71,TOR
707,SG,SF,-0.507,10,NYK
...,...,...,...,...,...
449,C,C,1.000,119,BKN
450,PG,PG,1.000,143,LAC
456,SG,SG,1.000,84,LAC
474,C,C,1.000,172,LAC


In [175]:
x.columns = ['p1','p2','corr','sample','team']
x = x.loc[abs(x['corr']) < .9]
x

,p1,p2,corr,sample,team
632,SG,SF,-0.555,71,TOR
636,SF,SG,-0.555,71,TOR
707,SG,SF,-0.507,10,NYK
711,SF,SG,-0.507,10,NYK
204,PG,C,-0.384,106,HOU
...,...,...,...,...,...
163,SF,PF,0.550,5,IND
545,C,PG,0.586,30,NOP
529,PG,C,0.586,30,NOP
166,PF,SG,0.670,7,IND


In [176]:
x.groupby(['p1','p2']).apply(lambda x: np.average(x['corr'], weights=x['sample'])).sort_values().reset_index()

,p1,p2,0
0,C,PF,-0.067
1,PF,C,-0.067
2,SF,C,-0.032
3,C,SF,-0.032
4,PF,PG,-0.031
...,...,...,...
15,SF,SG,0.002
16,PG,SG,0.002
17,SG,PG,0.002
18,C,PG,0.023


In [177]:
x.loc[x.team=='GSW']

,p1,p2,corr,sample,team
484,SG,C,-0.148,59,GSW
496,C,SG,-0.148,59,GSW
486,SF,SG,-0.132,63,GSW
482,SG,SF,-0.132,63,GSW
490,PF,PG,-0.017,216,GSW
...,...,...,...,...,...
479,PG,C,0.087,98,GSW
497,C,SF,0.100,88,GSW
489,SF,C,0.100,88,GSW
477,PG,SF,0.120,109,GSW
